# Tutorial 1: Simple quantum circuits

## Prerequisite: installation of myqlm

    pip install myqlm

    pip install myqlm-simulators

## Documentation

You will find code examples on https://myqlm.github.io.

## 1. Bell state circuit

Our aim is to prepare the Bell state $$\frac{|00\rangle + |11\rangle}{\sqrt{2}}$$


In [ ]:
from qat.lang.AQASM import H, CNOT, Program

# write a quantum program to construct a Bell state
#...

#circ = prog.to_circ()

#%qatdisplay circ

We now want to execute this circuit on a Quantum Processing Unit (QPU):

In [ ]:
# we load a "QPU" (here, a simulator)
from qat.qpus import get_default_qpu
qpu = get_default_qpu()

# now, we construct a "job" to submit to this QPU
job = circ.to_job(nbshots=10, aggregate_data=False)
res = qpu.submit(job)

The number of shots is the number of independent final $Z$-axis measurements on the quantum state. Thus, we expect to see the 10 outcomes of measuring both qubits on the $Z$ axis. 

We print the results:

In [ ]:
for sample in res:
    print(sample.state)

We can tell the QPU to aggregate the data, i.e collect the statistics of the outcomes to compute the histogram of the shots, and hence the estimated probability (``sample.probability``) (with its statistical error ``sample.err``) of a computational state in the final distribution:

In [ ]:
job = circ.to_job(nbshots=10, aggregate_data=False)
res = qpu.submit(job)
for sample in res:
    print(sample.state, sample.probability, sample.err)

Finally, since we are performing classical simulation, we have access to the exact probabilities, and to the probability amplitudes of the states (because we are doing pure-state simulations---as opposed to density-matrix simulations---at this stage). This is achieved by choosing an infinite number of shots, which we choose, by convention, by setting ``nbshots=0``:

In [ ]:
job = circ.to_job(nbshots=0)
res = qpu.submit(job)
for sample in res:
    print(sample.state, sample.probability, sample.amplitude)

## 2. Teleportation

Alice wants to send a qubit in state $|\psi\rangle$ to Bob with the help of a shared entangled pair between Alice and Bob.



### Solution
We suppose Alice has a state $\cos(\alpha/2) |0\rangle - i \sin(\alpha/2) |1\rangle$, and teleport it to Bob.

In [ ]:
from qat.lang.AQASM import RX, X, Z

alpha = 0.45

prog = Program()
reg = prog.qalloc(3) # 1 qubit for Alice, 1 qubit for Bob, 1 shared qubit
creg = prog.calloc(2) # 2 classical bits
# prepare state on Alice's side (unknown to Bob)
prog.apply(RX(alpha), reg[0])

# prepare Bell state
# ...

# teleportation circuit
# Hint: to perform an intermediate measurement: prog.measure(...)
# Hint: to perform an operation controlled by a classical bit: prog.cc_apply(...)

#...

#circ = prog.to_circ()

#%qatdisplay circ

Let us now execute the circuit, supposing we have infinite statistics (infinite number of shots):

In [ ]:
qpu = get_default_qpu()
# job = ... 
# res = ...

#for sample in res:
#    print(sample.state, sample.amplitude, sample.probability)

In [ ]:
import numpy as np
# Expected result
# RX(alpha) |0> = cos(alpha/2) - i sin(alpha/2)
print("Expecting: %s |0> - i %s |1>"%(np.cos(alpha/2), np.sin(alpha/2)))